In [1]:
import numpy as np

In [2]:
# split a univariate sequence into samples

def split_sequences(sequences, n_steps_in, n_steps_out):
    X = list()
    Y = list()
    
    for i in range(len(sequences)):
        
        # find the end of this pattern
        end_ix = i + n_steps_in #(0+3, 1+3, 2+3)
        out_end_ix = end_ix + n_steps_out #(3+2, 4+2, 5+2)
        
        # check if we are beyond the dataset 
        if out_end_ix > len(sequences):
            break
            
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix]
        seq_y = sequences[end_ix:out_end_ix]
        X.append(seq_x)
        Y.append(seq_y)
    return np.array(X), np.array(Y)

In [5]:
# define input sequence 
sequence = [10, 20, 30, 40, 50, 60, 70, 80]

In [6]:
n_steps = 3
n_steps_out = 2

In [7]:
X, Y = split_sequences(sequence, n_steps, n_steps_out)

In [8]:
X

array([[10, 20, 30],
       [20, 30, 40],
       [30, 40, 50],
       [40, 50, 60]])

In [9]:
Y

array([[40, 50],
       [50, 60],
       [60, 70],
       [70, 80]])

In [10]:
X.shape

(4, 3)

In [11]:
Y.shape

(4, 2)

In the case of the Encoder-Decoder model, the output, or y part, of the training dataset must also have this shape.

This is because the model will predict a given number of time steps with a given number of features for each input sample.

In [12]:
n_features = 1

In [13]:
X = X.reshape(X.shape[0], X.shape[1], n_features)
Y = Y.reshape(Y.shape[0], Y.shape[1], n_features)

In [14]:
X

array([[[10],
        [20],
        [30]],

       [[20],
        [30],
        [40]],

       [[30],
        [40],
        [50]],

       [[40],
        [50],
        [60]]])

In [15]:
Y

array([[[40],
        [50]],

       [[50],
        [60]],

       [[60],
        [70]],

       [[70],
        [80]]])

In [16]:
X.shape

(4, 3, 1)

In [17]:
Y.shape

(4, 2, 1)

# Model Building

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

Using TensorFlow backend.


# Encoder-Decoder Model

A model specifically developed for forecasting variable length output sequences is called the Encoder-Decoder LSTM.

The model was designed for prediction problems where there are both input and output sequences, so-called sequence-to-sequence, or seq2seq problems, such as translating text from one language to another.

As its name suggests, the model is comprised of two sub-models: the encoder and the decoder.

The encoder is a model responsible for reading and interpreting the input sequence. The output of the encoder is a fixed length vector that represents the model’s interpretation of the sequence.

The encoder is traditionally a Vanilla LSTM model, although other encoder models can be used such as Stacked, Bidirectional, and CNN models.

In [19]:
# define model
model = Sequential()
model.add(LSTM(100, activation = 'relu', input_shape = (n_steps, n_features)))

The decoder uses the output of the encoder as an input.

First, the fixed-length output of the encoder is repeated, once for each required time step in the output sequence.

In [20]:
model.add(RepeatVector(n_steps_out))

This sequence is then provided to an LSTM decoder model.

The model must output a value for each value in the output time step, which can be interpreted by a single output model

In [21]:
model.add(LSTM(100, activation = 'relu', return_sequences = True))

We can use the same output layer or layers to make each one-step prediction in the output sequence.

This can be achieved by wrapping the output part of the model in a TimeDistributed wrapper.

In [22]:
model.add(TimeDistributed(Dense(1)))

In [23]:
model.compile(optimizer = 'adam', loss = 'mse')

In [24]:
model.fit(X, Y, epochs = 100)


Epoch 1/100
4/4 [==============================] - 4s 1s/step - loss: 3766.8894
Epoch 2/100
4/4 [==============================] - 0s 8ms/step - loss: 3744.6543
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 3724.5874
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 3705.3867
Epoch 5/100
4/4 [==============================] - 0s 15ms/step - loss: 3686.5527
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 3667.3135
Epoch 7/100
4/4 [==============================] - 0s 21ms/step - loss: 3646.4827
Epoch 8/100
4/4 [==============================] - 0s 14ms/step - loss: 3623.2058
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 3596.8623
Epoch 10/100
4/4 [==============================] - 0s 14ms/step - loss: 3566.2812
Epoch 11/100
4/4 [==============================] - 0s 12ms/step - loss: 3530.4929
Epoch 12/100
4/4 [==============================] - 0s 4ms/step - loss: 3487.9365
Epoch 13/100
4/4 [

Epoch 100/100
4/4 [==============================] - 0s 6ms/step - loss: 6.2653


# Predictions

In [25]:
x_input = np.array([70, 80, 90])

In [26]:
x_input = x_input.reshape((1, n_steps, n_features))

In [27]:
model.predict(x_input)

array([[[ 94.42595],
        [110.38417]]], dtype=float32)